In [3]:
from model.classifier.hyde_similarity_classifier import HydeSimilarityClassifier
from model.business_object.crawl_session import CrawlSession
from model.business_object.page import Page
import pandas as pd

In [4]:
path_to_crawlers = "/Users/apollineguerineau/Documents/ENSAI/3A/Greece/internship/eval/ML_MOF_Diffusion/crawlers_v1/"
# Charger les deux fichiers CSV
csv1_path = path_to_crawlers + "seed_query_based_sim_cos_thr_0.802_pages_50/fetched_pages.csv"
csv2_path = path_to_crawlers + "seed_query_based_hyde_sim_cos_thr_0.882_pages_50/fetched_pages.csv"

csv1 = pd.read_csv(csv1_path)
csv2 = pd.read_csv(csv2_path)

merged_csv = pd.merge(
    csv1,
    csv2[["url", "score"]],  # Garder uniquement 'url' et 'score' du deuxième CSV
    on="url",
    how="left"
)

# Renommer les colonnes 'score' respectives
merged_csv.rename(columns={"score_x": "score_sim", "score_y": "score_hyde"}, inplace=True)

# Sauvegarder le résultat si nécessaire
merged_csv_path = path_to_crawlers + "merged_csv.csv"
merged_csv.to_csv(merged_csv_path, index=False)
print(merged_csv.head())



                                                 url  \
0  https://chemrxiv.org/engage/chemrxiv/article-d...   
1  https://chemrxiv.org/engage/chemrxiv/article-d...   
2  https://chemrxiv.org/engage/chemrxiv/article-d...   
3  https://chemrxiv.org/engage/chemrxiv/article-d...   
4  https://chemrxiv.org/engage/chemrxiv/article-d...   

                                               title  \
0  Inverse Design of ZIFs through Artificial Inte...   
1  Exploring Propane/Propylene Separation through...   
2  Towards a Generalizable Machine-Learned\nPoten...   
3  Molecular diffusion enhanced performance evalu...   
4  Machine Learning-driven models for predicting ...   

                                         description  score_sim  \
0  Artificial Intelligence (AI) benefits research...   0.888876   
1  Propylene is a vital component for the petroch...   0.865408   
2  Machine-learned Potentials (MLPs) have transfo...   0.864718   
3  Molecular diffusion is a fundamental property ...   0.8

In [9]:
# Compter le nombre de lignes avec NaN dans la colonne 'score_hyde'

classifier = HydeSimilarityClassifier(name_embed_model='intfloat/multilingual-e5-base')
crawl_session = CrawlSession(session_name='', query='')
crawl_session.hyde = {
    'title': 'Enhancing Adsorption Capacities of Metal-Organic Frameworks through Deep Learning and Diffusivity Optimization',
    'abstract': 'The discovery and optimization of metal-organic frameworks (MOFs) has been revolutionized by the integration of machine learning (ML) techniques with diffusivity modeling. This research aims to investigate the application of deep learning algorithms in predicting and optimizing the diffusivity properties of MOFs, leading to enhanced adsorption capacities. A comprehensive review of existing literature on ML-driven MOF design is conducted, followed by the development of a novel framework for integrating ML models with Monte Carlo simulations to predict diffusivity. The proposed framework is then validated using experimental data from various MOF materials, resulting in significant improvements in predicted and measured diffusivity values. The findings of this study have far-reaching implications for the design and optimization of COFs and ZIFs, enabling the creation of high-performance adsorbents for energy storage, gas separation, and catalysis applications.'
}

missing_score_hyde = merged_csv[merged_csv['score_hyde'].isna()]
print(f"Nombre de lignes sans score initialement : {len(missing_score_hyde)}")

for index, row in missing_score_hyde.iterrows():
    print(f"Traitement de l'index {index}")
    page = Page(url=row['url'], title=row['title'], description=row['description'])
    score = classifier.attribute_score(crawl_session=crawl_session, page=page)
    
    # Mettre à jour le score dans merged_csv directement
    merged_csv.loc[index, 'score_hyde'] = score

# Sauvegarder le dataframe mis à jour
merged_csv.to_csv(path_to_crawlers + "merged_csv_with_score.csv", index=False)

# Vérification des valeurs manquantes
remaining_missing = merged_csv['score_hyde'].isna().sum()
print(f"Nombre de lignes sans score après mise à jour : {remaining_missing}")



Nombre de lignes sans score initialement : 15
Traitement de l'index 25288
Traitement de l'index 26122
Traitement de l'index 26264
Traitement de l'index 26379
Traitement de l'index 26442
Traitement de l'index 26449
Traitement de l'index 26516
Traitement de l'index 26581
Traitement de l'index 27069
Traitement de l'index 27105
Traitement de l'index 27205
Traitement de l'index 27224
Traitement de l'index 27251
Traitement de l'index 27252
Traitement de l'index 27258
Nombre de lignes sans score après mise à jour : 0


In [27]:
# merged_csv.rename(columns={'score_hyde': 'score'}, inplace=True)

# # Supprimer la colonne 'score_sim'
# merged_csv.drop(columns=['score_sim'], inplace=True)

# Afficher les 5 premières lignes pour vérifier les modifications
merged_csv.head()

merged_csv.to_csv(path_to_crawlers + "seed_query_based_hyde_sim_cos_thr_0.882_pages_50/fetched_pages.csv", index=False, sep=',')

In [42]:
csv_test = merged_csv.head()
print(len(csv_test))
csv_test.to_csv(path_to_crawlers + "seed_query_based_hyde_sim_cos_thr_0.882_pages_50/fetched_pages.csv", index=False, sep=',')

5


In [43]:
import csv

# Chemin du fichier de sortie
output_file = path_to_crawlers + "seed_query_based_hyde_sim_cos_thr_0.882_pages_50/fetched_pages.csv"

# Colonnes à inclure dans le fichier CSV
columns = merged_csv.columns.tolist()

# Ouvrir le fichier en mode écriture
with open(output_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=columns)
    
    # Écrire l'en-tête
    writer.writeheader()
    
    # Écrire chaque ligne
    for _, row in merged_csv.iterrows():
        writer.writerow(row.to_dict())

In [45]:
df = pd.read_csv(path_to_crawlers + "seed_query_based_hyde_sim_cos_thr_0.882_pages_50/fetched_pages.csv", sep=',')
df.head()

,url,title,description,get_with_query,time_fetch,is_seed,score
0,https://chemrxiv.org/engage/chemrxiv/article-d...,Inverse Design of ZIFs through Artificial Inte...,Artificial Intelligence (AI) benefits research...,"""Machine Learning"" AND (diffusion OR diffusivi...",2024-11-20 15:30:39.428983,False,0.934894
1,https://chemrxiv.org/engage/chemrxiv/article-d...,Exploring Propane/Propylene Separation through...,Propylene is a vital component for the petroch...,"""Machine Learning"" AND (diffusion OR diffusivi...",2024-11-20 15:30:39.614658,False,0.923098
2,https://chemrxiv.org/engage/chemrxiv/article-d...,Towards a Generalizable Machine-Learned\nPoten...,Machine-learned Potentials (MLPs) have transfo...,"""Machine Learning"" AND (diffusion OR diffusivi...",2024-11-20 15:30:39.768251,False,0.915012
3,https://chemrxiv.org/engage/chemrxiv/article-d...,Molecular diffusion enhanced performance evalu...,Molecular diffusion is a fundamental property ...,"""Machine Learning"" AND (diffusion OR diffusivi...",2024-11-20 15:30:39.908374,False,0.935718
4,https://chemrxiv.org/engage/chemrxiv/article-d...,Machine Learning-driven models for predicting ...,This study advances the discourse on the appli...,"""Machine Learning"" AND (diffusion OR diffusivi...",2024-11-20 15:30:40.054033,False,0.932324


In [12]:
df = pd.read_csv("/Users/apollineguerineau/Documents/ENSAI/3A/Greece/internship/eval/ML_MOF_Diffusion/crawlers_v1/fetched_pages.csv", sep=',')
df.columns

ParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.
